In [ ]:
# MNIST 데이터 로딩
import torchvision
from torchvision import transforms
import torch


# 이미지 파일을 다운로드 : 지정도리 위치 지정=> ./ : 현재 디렉토리
image_path = './'

# 텐서 변환 , 0~1 정규화 됨
transform = transforms.Compose([transforms.ToTensor()])


mnist_dataset = torchvision.datasets.MNIST(root=image_path,
                                           train=True, #학습데이터터
                                           transform=transform,
                                           download=True)


from torch.utils.data import Subset
mnist_valid_dataset = Subset(mnist_dataset, torch.arange(10000))
mnist_train_dataset = Subset(mnist_dataset, torch.arange(10000, len(mnist_dataset)))
mnist_test_dataset = torchvision.datasets.MNIST(root=image_path,
                                           train=False, #태수투 데이터
                                           transform=transform,
                                           download=False)

Failed to download (trying next):
HTTP Error 404: Not Found



100.0%


Extracting ./MNIST\raw\train-images-idx3-ubyte.gz to ./MNIST\raw

Failed to download (trying next):
HTTP Error 404: Not Found



100.0%


Extracting ./MNIST\raw\train-labels-idx1-ubyte.gz to ./MNIST\raw

Failed to download (trying next):
HTTP Error 404: Not Found



100.0%


Extracting ./MNIST\raw\t10k-images-idx3-ubyte.gz to ./MNIST\raw

Failed to download (trying next):
HTTP Error 404: Not Found



100.0%

Extracting ./MNIST\raw\t10k-labels-idx1-ubyte.gz to ./MNIST\raw



In [2]:
from torch.utils.data import DataLoader


batch_size = 64
torch.manual_seed(1)
train_dl = DataLoader(mnist_train_dataset, batch_size, shuffle=True)
valid_dl = DataLoader(mnist_valid_dataset, batch_size, shuffle=False)

In [5]:
# cnn 신경망 구성 
import torch.nn as nn

# 계산그래프 정의
model=nn.Sequential()
# 첫번쨰 cnn 층을 추가 :입력받는 층이고, 출력은 out_channels로 결정된다다
model.add_module(
    'conv1', #레퍼런스 변수수
    nn.Conv2d(  #입력정의 
        in_channels=1, #색상 : 흑백 
        out_channels=32, #필터 (커널) 개수
        kernel_size=5, # (5,5)
        padding=2 #zero padding 
        )
)
# 활성화 함수 지정
model.add_module('relu1', nn.ReLU()) #가중치없다, 변환해주는 함수다

#  폴링층 설정
model.add_module('pool1', nn.MaxPool2d(kernel_size=2)) #2-> (2,2)

# 두번째 cnn 층 설정
model.add_module(
    'conv2',
    nn.Conv2d(
        in_channels=32,
        out_channels=64,
        kernel_size=5,
        padding=2
    )
)
model.add_module('relu2', nn.ReLU())
model.add_module('pool2', nn.MaxPool3d(kernel_size=2))

model


Sequential(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (relu1): ReLU()
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (relu2): ReLU()
  (pool2): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)

In [6]:
# x-> 다 1들어있는 임의 데이터
x=torch.ones((4,1,28,28)) #4 : batch_size
model(x).shape

torch.Size([4, 32, 7, 7])

In [8]:
# flatten layer create
model.add_module('flatten', nn.Flatten())
model

Sequential(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (relu1): ReLU()
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (relu2): ReLU()
  (pool2): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
)

In [9]:
x=torch.ones([4,1,28,28])
# torch.size([]4, 64, 7, 7))
# flatten
model(x).shape

torch.Size([4, 1568])

In [10]:
# 분류에 사용되는 선형신경망을 2개 추가 : 10진분류 (output 이 10개여야 한다)
model.add_module('linear1', nn.Linear(in_features=3136, out_features=1024))#2의 10승
model.add_module('relu3', nn.ReLU())
model.add_module('dropout1', nn.Dropout(p=0.5)) #신경망에서의 규제(일반화성능을 높이기위해)
# 마지막 출력 선형 신경망을 추가
model.add_module('linear2', nn.Linear(1024,10))
model

Sequential(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (relu1): ReLU()
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (relu2): ReLU()
  (pool2): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear1): Linear(in_features=3136, out_features=1024, bias=True)
  (relu3): ReLU()
  (dropout1): Dropout(p=0.5, inplace=False)
  (linear2): Linear(in_features=1024, out_features=10, bias=True)
)

In [12]:
import torch
import torch.nn as nn

model = nn.Sequential(
    nn.Conv2d(1, 32, kernel_size=5, padding=2),  # (1, 28, 28) → (32, 28, 28)
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2),                # (32, 14, 14)
    
    nn.Conv2d(32, 64, kernel_size=5, padding=2), # (64, 14, 14)
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2),                # (64, 7, 7)
    
    nn.Flatten(),                               # (64 * 7 * 7) = 3136
    nn.Linear(3136, 128),                        # ⚠️ 3136 입력 필요
    nn.ReLU(),
    nn.Linear(128, 10)                           # 출력 클래스 수: 예시로 10개
)


In [13]:
x = torch.ones([4, 1, 28, 28])  # 배치 크기: 4
out = model(x)
print(out.shape)  # 👉 torch.Size([4, 10])


torch.Size([4, 10])


In [ ]:
model_device=next(model.parameters()).device
print(f'모델이 위치한 디바이스 : {model_device}')


# MNIST 데이터의 디바이스 확인
data_device=mnist_test_dataset.data.device
print(f'MNIST 데이터가 위치한 디바이스 : {data_device}')

모델이 위치한 디바이스 : cpu
MNIST 데이터가 위치한 디바이스 : cpu


In [17]:
# model 을 gpu로 보낵기
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'현재 사용 가능한 장치 : {device}')

# 모델 보내기
model=model.to(device)
# 모델이 어디 있는지 확인 코드드
model_device=next(model.parameters()).device
print(f'모델이 위치한 디바이스 : {model_device}')


현재 사용 가능한 장치 : cpu
모델이 위치한 디바이스 : cpu


In [ ]:
# 학습을 진행하는 함수 
# num_epochs, train_dl, valid_dl, model, loss_fn, optimizer
def train():#함수가 돌아가는 데 필요한 재료 : optimizer,train data, test data, 
    # 에포크 개수 반복 : num_epochs
    # 예측값 추출 : model (학습데이터):train_dl
    # 오차 : loss_fn (예측값, 원래답)
    # 최적화 : otimizer
    # 1에폭마다 평가 : valid data, valid data label -> valid_dl
    # return
    # 학습데이터에 대한 오차, 검증데이터 오차, 학습데이터 정확도, 검증데이터 정확도도
    def train(model, num_epochs, train_dl, valid_dl, loss_fn, optimizer):
        loss_hist_train=[0] * num_epochs #cpu
        accuracy_hist_train=[0] * num_epochs #cpu 
        loss_hist_valid[0] * num_epochs #cpu
        accuracy_hist_valid[0] * num_epochs #cpu
        
        for epoch in range(num_epochs):
            # 모델을 학습 모드로 설정
            model.train()
            for x_batch, y_batch in train_dl: #100, 10 : 10번반복
                x_batch=x_batch.to(device) #gpu로 보내라
                y_batch=y_batch.to(device) #gpu로 보내라
                pred=model(x_batch) #예측값 추출(배치만큼)
                # pred 는 gpu 
                loss=loss_fn(pred, y_batch)
                loss.backward() #가중치로 미분, 오차가 최소가 되는 
                optimizer.step() # 배치만큼 가중치 수정
                optimizer.zero_grad() #가중치 초기화
                 
                # 시각화 데이터 추출
                # 전체 오차 누적 저장
                loss_hist_train[epoch] += loss.item()*y_batch.size(0) #평균값 
                # 누적 정학도 : 배치사이즈만큼 나온다다
                is_correct=(torch.argmax(pred,dim=1)==y_batch).float()
                accuracy_hist_train[epoch] +=is_correct.sum().cpu()
            
            # 1. epoch마다 평균오차, 평균 정확도
            loss_hist_train[epoch] /= len(train_dl.dataset) #dataset의 lenght는 100
            accuracy_hist_train[epoch] /= len(train_dl.dataset)
             
            # 모델을 평가 모드로 살장 : 결과를 위 리스트에 대입
            model.eval() 
            with torch.no_grad():
                for x_batch, y_batch in valid_dl:
                    x_batch=x_batch.to(device)
                    y_batch=y_batch.toi(device)
                    
                    pred=model(x_batch) #예측값 = 원래값
                    loss=loss_fn(pred,y_batch)
                    loss_hist_valid[epoch] += loss.item()*y_batch.size(0)
                    
                    is_correct = (torch.argmaz(pred.dim=1)==y_batch).float()
                    accuracy_hist_valid += is_correct.sum().cpu()
                    # 여기까지는 1배치만큼씩 처리
                    
            loss_hist_valid[epoch] /= len(valid_dl.dataset)
            accuracy_hist_valid[epoch] /= len(valid_dl.dataset)
            
            # 학습데이터 정확도와 검증데이터의 정확도 출력
            print9(f'애포크 : {epoch}, 학습정확도 : {accuracy_hist_train{epoch}.4f}
                   검증정확도 : {accuracy_hist_valid{epoch}.4f}')
            return loss_hist_train, loss_hist_valid, accuracy_hist_train, accuracy_hist_valid
                    
    # train data, test data 
    

SyntaxError: invalid syntax (654422144.py, line 20)

In [20]:
# 학습을 진행하는 함수
# num_epochs, train_dl, vaild_dl, model, loss_fn, optimizer
# epoch 개수 반복 : num_epochs
# 예측값 추출 : model(학습데이터) : train_dl
# 오차 : loss_fn(예측값, 원래답데이터)
# # 최적화 : optimizer
# 1에폭마다 평가 : valid data + valid data label => valid_dl
# return 에포마다
# 학습데이터에 대한 오차, 검증 데이터 오차, 학습데이터 정확도, 검증 정확도
def train(model, num_epochs, train_dl, valid_dl, loss_fn, optimizer):
    loss_hist_train=[0] * num_epochs # cpu
    accuracy_hist_train=[0] * num_epochs # cpu
    loss_hist_valid=[0] * num_epochs # cpu
    accuracy_hist_valid=[0] * num_epochs # cpu
   
    for epoch in range(num_epochs):
        # 모델을 학습 모드로 설정
        model.train()
        for x_batch, y_batch in train_dl: # 100 , 10 : 10번 반복
            x_batch=x_batch.to(device) # gpu로 보내라
            y_batch=y_batch.to(device) # gpu로 보내라
            pred=model(x_batch) # 예측값 추출 (배치만큼)
            loss=loss_fn(pred, y_batch)
            loss.backward() # 가중치로 미분, 오차가 최소가 되는
            optimizer.step() # 가중치 수정
            optimizer.zero_grad() # 가중치 초기화
           
            # 시각화 데이터 추출
            # 전체 오차 누적 저장
            loss_hist_train[epoch] += loss.item()*y_batch.size(0)
            # 누적 정확도 : 배치사이즈만큼 나온다
            is_correct=(torch.argmax(pred,dim=1)==y_batch).float()
            accuracy_hist_train[epoch] +=is_correct.sum().cpu()
        # 1 epoch마다 평균 오차, 평균 정확도
        loss_hist_train[epoch] /= len(train_dl.dataset)
        accuracy_hist_train[epoch] /= len(train_dl.dataset)
       
        # 모델을 평가 모드로 설정: 결과 위 리스트 대입
        model.eval()
        with torch.no_grad():
            for x_batch, y_batch in valid_dl:
                x_batch=x_batch.to(device)
                y_batch=y_batch.to(device)
               
                pred=model(x_batch) # 예측값 - 원래값값
                loss=loss_fn(pred,y_batch)
                loss_hist_valid[epoch] += loss.item()*y_batch.size(0)
                is_correct = (torch.argmax(pred,dim=1)==y_batch).float()
                accuracy_hist_valid[epoch] += is_correct.sum().cpu()
                # 여기까지는 1배치만큼씩 처리
       
        loss_hist_valid[epoch] /= len(valid_dl.dataset)        
        accuracy_hist_valid[epoch] /= len(valid_dl.dataset)
       
        # 학습데이터 정확도와 검증데이터의 정확도 출력
        print(f'에포크 : {epoch+1},학습정확도 : {accuracy_hist_train[epoch]:.4f}, 검증정확도 : {accuracy_hist_valid[epoch]:.4f} ')
    return loss_hist_train, loss_hist_valid, accuracy_hist_train, accuracy_hist_valid 

In [21]:
# loss_fn, optimizer
loss_fn=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(), lr=0.001)
num_epochs=200

hist=train(model, num_epochs, train_dl, valid_dl, loss_fn, optimizer)

에포크 : 1,학습정확도 : 0.9484, 검증정확도 : 0.9824 
에포크 : 2,학습정확도 : 0.9845, 검증정확도 : 0.9854 
에포크 : 3,학습정확도 : 0.9896, 검증정확도 : 0.9859 
에포크 : 4,학습정확도 : 0.9923, 검증정확도 : 0.9875 
에포크 : 5,학습정확도 : 0.9943, 검증정확도 : 0.9885 
에포크 : 6,학습정확도 : 0.9954, 검증정확도 : 0.9855 
에포크 : 7,학습정확도 : 0.9957, 검증정확도 : 0.9875 
에포크 : 8,학습정확도 : 0.9969, 검증정확도 : 0.9907 
에포크 : 9,학습정확도 : 0.9976, 검증정확도 : 0.9898 
에포크 : 10,학습정확도 : 0.9971, 검증정확도 : 0.9859 
에포크 : 11,학습정확도 : 0.9979, 검증정확도 : 0.9890 
에포크 : 12,학습정확도 : 0.9984, 검증정확도 : 0.9881 
에포크 : 13,학습정확도 : 0.9984, 검증정확도 : 0.9910 
에포크 : 14,학습정확도 : 0.9988, 검증정확도 : 0.9895 
에포크 : 15,학습정확도 : 0.9984, 검증정확도 : 0.9902 
에포크 : 16,학습정확도 : 0.9985, 검증정확도 : 0.9899 
에포크 : 17,학습정확도 : 0.9986, 검증정확도 : 0.9888 
에포크 : 18,학습정확도 : 0.9989, 검증정확도 : 0.9892 
에포크 : 19,학습정확도 : 0.9985, 검증정확도 : 0.9903 
에포크 : 20,학습정확도 : 0.9990, 검증정확도 : 0.9904 
에포크 : 21,학습정확도 : 0.9993, 검증정확도 : 0.9903 
에포크 : 22,학습정확도 : 0.9991, 검증정확도 : 0.9891 
에포크 : 23,학습정확도 : 0.9987, 검증정확도 : 0.9892 
에포크 : 24,학습정확도 : 0.9994, 검증정확도 : 0.9918 
에포크 : 25,학습정확도 : 0.9992, 

In [ ]:
import matplotlib.pyplot as plt

x_arr = np.arange(len(hist[0])) + 1


fig = plt.figure(figsize=(12, 4))
ax = fig.add_subplot(1, 2, 1)
ax.plot(x_arr, hist[0], '-o', label='Train loss')
ax.plot(x_arr, hist[1], '--<', label='Validation loss')
ax.set_xlabel('Epoch', size=15)
ax.set_ylabel('Loss', size=15)
ax.legend(fontsize=15)
ax = fig.add_subplot(1, 2, 2)
ax.plot(x_arr, hist[2], '-o', label='Train acc.')
ax.plot(x_arr, hist[3], '--<', label='Validation acc.')
ax.legend(fontsize=15)
ax.set_xlabel('Epoch', size=15)
ax.set_ylabel('Accuracy', size=15)


#plt.savefig('figures/14_13.png')
plt.show()

NameError: name 'np' is not defined

In [ ]:
# 일반화 성능 :test data 예측값 추출 : 원래 비교 정확도]
# mnist_test.dataset : cpu, modle : gpu
# 추로할떄는 cpu 로 넘기는 것이 좋다 
# gpu 연산이이 종료될 떄까지 기다리게 만들어야 한다
torch.cuda.syncronize=() 
model.cpu()
pred=model(mnist_test_dataset.data.unsqueeze(1)/255.)

is_correct=(torch.argmax(pred,dim=1)==mnist_train_dataset.targets).float()
print(f'테스트 정확도 : {is_correct.mean():.4f}')

In [ ]:
fig = plt.figure(figsize=(12, 4))
for i in range(12):
    ax = fig.add_subplot(2, 6, i+1)
    ax.set_xticks([]); ax.set_yticks([])
    img = mnist_test_dataset[i][0][0, :, :]
    pred = model(img.unsqueeze(0).unsqueeze(1))
    y_pred = torch.argmax(pred)
    ax.imshow(img, cmap='gray_r')
    ax.text(0.9, 0.1, y_pred.item(),
            size=15, color='blue',
            horizontalalignment='center',
            verticalalignment='center',
            transform=ax.transAxes)


#plt.savefig('figures/14_14.png')
plt.show()


In [ ]:
# 좋은 모델 생성 -> 저장 
import os

if not os.path.exists('models') #models 디렉토리가 존재하지 않으면 true 
path='models/mnist_cnn_09932_acc_250423.ph'
torch.save(model, path)